In [2]:
%matplotlib inline
import os
import numpy as np
import glob
import xarray as xr
import pdb
from pathlib import Path
import pandas as pd
import dask
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import sys
# import cartopy.crs as ccrs
from datetime import datetime
from matplotlib.colors import ListedColormap
from skimage.filters import sobel
from skimage import feature
from skimage.morphology import disk, binary_dilation, binary_erosion
from scipy import ndimage
from IPython.core.display import Video
import rioxarray
# from dea_tools.datahandling import paths_to_datetimeindex
sys.path.append('/g/data/jk72/sc0554/dea-notebooks/Tools/dea_tools/')
from plotting import rgb, xr_animation

In [ ]:
def paths_to_datetimeindex(paths, string_slice=(0, 10), form='%Y%m'):
    """
    Helper function to generate a Pandas datetimeindex object
    from dates contained in a file path string.
    Parameters
    ----------
    paths : list of strings
        A list of file path strings that will be used to extract times
    string_slice : tuple
        An optional tuple giving the start and stop position that
        contains the time information in the provided paths. These are
        applied to the basename (i.e. file name) in each path, not the
        path itself. Defaults to (0, 10).
    Returns
    -------
    A pandas.DatetimeIndex object containing a 'datetime64[ns]' derived
    from the file paths provided by `paths`.
    """
    date_strings = [os.path.basename(i)[slice(*string_slice)]
                    for i in paths]
    return pd.to_datetime(date_strings, format=form)

def add_time_dim(xda):
    xda = xda.expand_dims(time = [datetime.now()])
    return xda

## NSIDC Sea Ice Concentration

In [ ]:
# This command downloads the NSIDC data
#! wget -r --no-parent -A 'S_202*concentration_v3.0.tif' ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/south/monthly/geotiff/ -P /g/data/jk72/sc0554
path = '/g/data/jk72/MIZ/sidads.colorado.edu/'

geotiff_list = []

for file in Path(path).rglob('*.tif'):
    geotiff_list.append(file)

# geotiff_list = sorted(geotiff_list, key=lambda i: int(os.path.splitext(os.path.basename(i)[2:8])[0]))
print(len(geotiff_list))
xr.open_rasterio(geotiff_list[0]).plot()

In [ ]:
# Create variable used for time axis
time_var = xr.Variable('time', paths_to_datetimeindex(geotiff_list,
                                                      string_slice=(2, 8), form=))

# Load in and concatenate all individual GeoTIFFs
geotiffs_da = xr.concat([xr.open_rasterio(i) for i in geotiff_list],
                        dim=time_var)

# Covert our xarray.DataArray into a xarray.Dataset
geotiffs_ds = geotiffs_da.to_dataset('band')

# Rename the variable to a more useful name
geotiffs_ds = geotiffs_ds.rename({1: 'conc'})

# Print the output
print(geotiffs_ds)

## Bremen Sea Ice concentration

In [3]:
# filename = '/g/data/jk72/MIZ/processed/asi2_daily.nc'
# daily = xr.open_dataset(filename)
filename = '/g/data/jk72/MIZ/processed/asi2_monthly.nc'
monthly_resample = xr.open_dataset(filename)

In [ ]:
# Resample daily to monthly

monthly_resample = data.resample(time="M").mean(keep_attrs=True)
monthly_resample['conc'].mean(dim=['x', 'y']).plot()

# Calculate monthly average
# monthly_average = daily.resample(time="M").mean(keep_attrs=True)

In [ ]:
g = monthly_average['conc'].sel(time=slice('2020-01-01','2020-12-31')).plot(col='time', col_wrap=3, cmap='Blues_r', vmin=0, vmax=100)

for i, ax in enumerate(g.axes.flat):

    ax.set_title(f"Month {i+1}")
    
# plt.savefig('/g/data/jk72/sc0554/miz/bremen_monthly_average.png')

In [ ]:
plt.figure(figsize=(12,13))
plt.suptitle('Mean Sea Ice Concentration', fontsize=16)

for iMonth in np.arange(0, 12):
    
    ax = plt.subplot(4,4,iMonth+1,projection=ccrs.SouthPolarStereo())
    
    toPlot=monthly_average.sel(time=iMonth)
    
    plt.imshow(
        toPlot['conc'],
        #levels=[0,.15,.80,1.00],
#         levels=np.arange(0,1.04,.1),
        transform=ccrs.SouthPolarStereo(true_scale_latitude=-70),
        cmap='Blues_r',
        )

#     gl = ax.gridlines(
#         draw_labels=False, linewidth=2, color='gray', alpha=0.5, linestyle='--'
#     )

#     ax.coastlines()
    plt.title('Month ' + str(iMonth+1))
    
plt.axes
plt.colorbar(orientation='horizontal', cax = plt.axes([0.2,0.2,0.6,0.04]))
plt.show()

In [ ]:
# Produce time series animation
xr_animation(ds=geotiffs_ds, 
             bands=['conc'],
             output_path='animated_timeseries.mp4',                                
             interval=100, 
             width_pixels=300,
             show_date='%b %Y')

# Plot animation
plt.close()
Video('animated_timeseries.mp4', embed=True)

In [33]:
mask

<xarray.DataArray 'conc' (y: 1328, x: 1264)>
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
Coordinates:
    time     datetime64[ns] 2012-07-31
  * y        (y) float64 4.347e+06 4.341e+06 4.334e+06 ... -3.941e+06 -3.947e+06
  * x        (x) float64 -3.947e+06 -3.941e+06 ... 3.941e+06 3.947e+06

In [ ]:
# mask.to_netcdf('/g/data/jk72/sc0554/miz/processing/mask.nc')